In [35]:
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
import numpy as np
import random
from certifai.common.utils.encoding import CatEncoder
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from keras.models import Sequential
from keras import regularizers
from keras import optimizers
from keras.layers import Dense, Dropout
import os
import random
import tensorflow as tf
seed = 42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_random_seed(seed)

base_path = '..'
all_data_file = f"{base_path}/datasets/german_credit_eval.csv"

df = pd.read_csv(all_data_file)

cat_columns = [
    'checkingstatus',
    'history',
    'purpose',
    'savings',
    'employ',
    'status',
    'others',
    'property',
    'age',
    'otherplans',
    'housing',
    'job',
    'telephone',
    'foreign'
    ]

label_column = 'outcome'

# Separate outcome
y = df[label_column]
X = df.drop(label_column, axis=1)

encoder = CatEncoder(cat_columns, X)

# NN separate for probability scoring label 
y_model = np.asarray([(1,0) if x==1 else (0,1) for x in y ])

best_model_path = os.path.join('.', 'best_model_keras')
es = EarlyStopping(monitor='accuracy', mode='min', verbose=1,
                   patience=100, min_delta=0.0001)

rlp = ReduceLROnPlateau(monitor='accuracy', factor=0.02, patience=20, verbose=1, mode='min',
                        min_delta=0.001, cooldown=1, min_lr=0.0001)
mcp = ModelCheckpoint(best_model_path, monitor='accuracy', verbose=1,
                      save_best_only=True, save_weights_only=False, mode='max')

model = Sequential()
model.add(Dense(60, input_dim=encoder(X.values).shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(encoder(X.values), y_model, epochs=100, batch_size=10, callbacks=[mcp,rlp,es],validation_split=0.2)
_, accuracy = model.evaluate(encoder(X.values), y_model)
print('Accuracy: %.2f' % (accuracy*100))

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 1s 633us/step - loss: 0.5866 - accuracy: 0.7038 - val_loss: 0.5489 - val_accuracy: 0.7325

Epoch 00001: accuracy improved from -inf to 0.70375, saving model to ./best_model_keras
Epoch 2/100
800/800 [==============================] - 0s 190us/step - loss: 0.5167 - accuracy: 0.7337 - val_loss: 0.4970 - val_accuracy: 0.7675

Epoch 00002: accuracy improved from 0.70375 to 0.73375, saving model to ./best_model_keras
Epoch 3/100
800/800 [==============================] - 0s 182us/step - loss: 0.4751 - accuracy: 0.7681 - val_loss: 0.4760 - val_accuracy: 0.7850

Epoch 00003: accuracy improved from 0.73375 to 0.76812, saving model to ./best_model_keras
Epoch 4/100
800/800 [==============================] - 0s 172us/step - loss: 0.4514 - accuracy: 0.7944 - val_loss: 0.4787 - val_accuracy: 0.7675

Epoch 00004: accuracy improved from 0.76812 to 0.79438, saving model to ./best_model_keras
Epoch 5/1

800/800 [==============================] - 0s 119us/step - loss: 0.1204 - accuracy: 0.9725 - val_loss: 0.6338 - val_accuracy: 0.7750

Epoch 00036: accuracy did not improve from 0.97500
Epoch 37/100
800/800 [==============================] - 0s 117us/step - loss: 0.1193 - accuracy: 0.9725 - val_loss: 0.6387 - val_accuracy: 0.7675

Epoch 00037: accuracy did not improve from 0.97500
Epoch 38/100
800/800 [==============================] - 0s 110us/step - loss: 0.1178 - accuracy: 0.9725 - val_loss: 0.6386 - val_accuracy: 0.7750

Epoch 00038: accuracy did not improve from 0.97500
Epoch 39/100
800/800 [==============================] - 0s 115us/step - loss: 0.1167 - accuracy: 0.9744 - val_loss: 0.6408 - val_accuracy: 0.7700

Epoch 00039: accuracy did not improve from 0.97500
Epoch 40/100
800/800 [==============================] - 0s 111us/step - loss: 0.1155 - accuracy: 0.9750 - val_loss: 0.6425 - val_accuracy: 0.7750

Epoch 00040: accuracy did not improve from 0.97500
Epoch 41/100
800/800 [=

800/800 [==============================] - 0s 109us/step - loss: 0.0785 - accuracy: 0.9875 - val_loss: 0.7149 - val_accuracy: 0.7775

Epoch 00075: accuracy did not improve from 0.98750
Epoch 76/100
800/800 [==============================] - 0s 114us/step - loss: 0.0775 - accuracy: 0.9875 - val_loss: 0.7206 - val_accuracy: 0.7700

Epoch 00076: accuracy did not improve from 0.98750
Epoch 77/100
800/800 [==============================] - 0s 110us/step - loss: 0.0769 - accuracy: 0.9881 - val_loss: 0.7186 - val_accuracy: 0.7825

Epoch 00077: accuracy improved from 0.98750 to 0.98813, saving model to ./best_model_keras
Epoch 78/100
800/800 [==============================] - 0s 118us/step - loss: 0.0761 - accuracy: 0.9887 - val_loss: 0.7222 - val_accuracy: 0.7750

Epoch 00078: accuracy improved from 0.98813 to 0.98875, saving model to ./best_model_keras
Epoch 79/100
800/800 [==============================] - 0s 110us/step - loss: 0.0751 - accuracy: 0.9894 - val_loss: 0.7244 - val_accuracy: 0.

In [36]:
# initial value counts of outcome label
df['outcome'].value_counts()

1    700
2    300
Name: outcome, dtype: int64

In [37]:
y_preds = model.predict(encoder(X.values))
y_classes = np.argmax(y_preds,axis=1)
y_actual = np.where(y==1, 0,1)
np.mean(y_actual == y_classes)

0.948

In [38]:
class_1_threshold = 0.95
class_2_threshold = 0.4

In [39]:
y_class_1 = y_preds[y_preds[:,0] > class_2_threshold][:,0]
y_class_1.shape

(718,)

In [40]:
y_class_2 = y_preds[y_preds[:,1] > class_1_threshold][:,1]
y_class_2.shape

(165,)

In [41]:
y_class_1 = y_preds[y_preds[:,0] > class_2_threshold][:,0]
np.sum(y_class_1 < class_1_threshold)

161

### creating new label

In [42]:
# loan_granted 1
new_label_1 = y_preds[y_preds[:,0] > class_1_threshold][:,0]
new_label_1.shape

(557,)

In [43]:
#loan_denied 2
new_label_2 = y_preds[y_preds[:,0] < class_2_threshold][:,0]
new_label_2.shape

(282,)

In [44]:
# further_inspection 3
new_label_3 = y_preds[(y_preds[:,0] > class_2_threshold) & (y_preds[:,0] < class_1_threshold)][:,0]
new_label_3.shape

(161,)

### arrange the new labels to be appended to dataset

In [45]:
new_label = []
for z in y_preds:
    prob_class_1 = z[0]
    if prob_class_1 > class_1_threshold:
        new_label.append(1)
    elif prob_class_1 < class_2_threshold:
        new_label.append(2)
    else:
        new_label.append(3)

In [46]:
np.mean(y==new_label)

0.81

### create dataset with new labels

In [47]:
df_new = df.copy()
df_new[label_column] = new_label

### check new outcome label counts

In [48]:
df_new['outcome'].value_counts()

1    557
2    282
3    161
Name: outcome, dtype: int64

### dump the new dataset as csv

In [49]:
save_as = f"{base_path}/datasets/german_credit_eval_multiclass.csv"
df_new.to_csv(save_as,index=False)

### check file is saved correctly

In [50]:
!ls $base_path/datasets | grep multiclass

german_credit_eval_multiclass.csv
